In [ ]:
#export
from local.torch_basics import *
from local.test import *
from local.core import *
from local.data.all import *
from local.tabular.core import *
from local.notebook.showdoc import show_doc

In [ ]:
#default_exp tabular.rapids

# Tabular with rapids

> Basic functions to preprocess tabular data before assembling it in a `DataBunch` on the GPU.

In [ ]:
#export
try: import cudf,nvcategory
except: print("This requires rapids, see https://rapids.ai/ for installation details")

In [ ]:
#export
@patch
def __array__(self:cudf.DataFrame): return self.pandas().__array__()

In [ ]:
class TabularGPU(Tabular):
    def transform(self, cols, f): 
        for c in cols: self.set_col(c, f(self.items[c]))
            
    def __getattr__(self,k):
        if isinstance(self.items, cudf.DataFrame) and k in self.items.columns: return self.items[k]
        return super().__getattr__(k)

## TabularProcessors

In [ ]:
#export
def _to_str(c): return c if c.dtype == "object" else c.astype("str")
def _remove_none(c):
    if None in c: c.remove(None)
    return c

In [ ]:
#export
@Categorify
def setups(self, to: TabularGPU):
    self.lbls = {n: nvcategory.from_strings(_to_str(to[:to.split,n]).data).keys() for n in to.all_cat_names}
    self.items = to.classes = {n: CategoryMap(_remove_none(c.to_host()), add_na=True) for n,c in self.lbls.items()}
   
@patch
def _apply_cats_gpu(self: Categorify, c): 
    return cudf.Series(nvcategory.from_strings(_to_str(c).data).set_keys(self.lbls[c.name]).values()).add(1)

@Categorify
def encodes(self, to: TabularGPU): to.transform(to.all_cat_names, self._apply_cats_gpu)

In [ ]:
cat = Categorify()
df = cudf.from_pandas(pd.DataFrame({'a':[0,1,2,0,2]}))
to = TabularGPU(df, 'a')

cat.setup(to)
test_eq(list(cat['a']), ['#na#','0','1','2'])
test_eq(to.a.to_array(), np.array([1,2,3,1,3]))
df1 = cudf.from_pandas(pd.DataFrame({'a':[1,0,3,-1,2]}))
to1 = TabularGPU(df1, 'a')
cat(to1)
#Values that weren't in the training df are sent to 0 (na)
test_eq(to1.a.to_array(), np.array([2,1,0,0,3]))

#Test decode
to2 = TabularPandas(df1.to_pandas(), 'a')
cat.decode(to2)
test_eq(to2.a, np.array(['1','0','#na#','#na#','2']))

In [ ]:
cat = Categorify()
df = cudf.from_pandas(pd.DataFrame({'a':[0,1,2,3,2]}))
to = TabularGPU(df, 'a', split=3)
cat.setup(to)
test_eq(list(cat['a']), ['#na#','0','1','2'])
test_eq(to.a.to_array(), np.array([1,2,3,0,3]))

In [ ]:
#TODO Categorical
cat = Categorify()
df = cudf.from_pandas(pd.DataFrame({'a':pd.Categorical(['M','H','L','M'], categories=['H','M','L'], ordered=True)}))
to = TabularGPU(df, 'a')
cat.setup(to)
#Fails for now
#test_eq(cat['a'].to_host(), ['H','M','L'])
#test_eq(df["a"].to_array(), [2,1,3,2])

<cudf.DataFrame ncols=1 nrows=4 >

In [ ]:
#export
@Normalize
def setups(self, to: TabularGPU):
    self.means = {n: to[:to.split,n].mean()           for n in to.cont_names}
    self.stds  = {n: to[:to.split,n].std(ddof=0)+1e-7 for n in to.cont_names}

@Normalize
def encodes(self, to: TabularGPU):
    to.transform(to.cont_names, lambda c: (c-self.means[c.name])/self.stds[c.name])

In [ ]:
norm = Normalize()
df = cudf.from_pandas(pd.DataFrame({'a':[0,1,2,3,4]}))
to = TabularGPU(df, cont_names='a')
norm.setup(to)
x = np.array([0,1,2,3,4])
m,s = x.mean(),x.std()
test_eq(norm.means['a'], m)
test_close(norm.stds['a'], s)
test_close(to.a.to_array(), (x-m)/s)
df1 = cudf.from_pandas(pd.DataFrame({'a':[5,6,7]}))
to1 = TabularGPU(df1, cont_names='a')
norm(to1)
test_close(to1.a.to_array(), (np.array([5,6,7])-m)/s)
to2 = TabularPandas(df1.to_pandas(), cont_names='a')
to2 = norm.decode(to2)
test_close(to2.a, [5,6,7])

In [ ]:
norm = Normalize()
df = cudf.from_pandas(pd.DataFrame({'a':[0,1,2,3,4]}))
to = TabularGPU(df, cont_names='a', split=3)
norm.setup(to)
x = np.array([0,1,2])
m,s = x.mean(),x.std()
test_eq(norm.means, {'a': m})
test_close(norm.stds['a'], s)
test_close(to.a.to_array(), (np.array([0,1,2,3,4])-m)/s)

In [ ]:
#export
@patch
def median(self:cudf.Series):
    "Get the median of `self`"
    col = self.dropna().reset_index(drop=True).sort_values()
    return col[len(col)//2] if len(col)%2 != 0 else (col[len(col)//2]+col[len(col)//2-1])/2

In [ ]:
col = cudf.Series([0,1,np.nan,1,2,3,4])
test_eq(col.median(), 1.5)
col = cudf.Series([np.nan,1,np.nan,1,2,3,4])
test_eq(col.median(), 2)

In [ ]:
#export
@FillMissing
def setups(self, to: TabularGPU):
    self.na_dict = {}
    for n in to.cont_names:
        col = to.loc[:to.split, n]
        if col.isnull().any(): self.na_dict[n] = self.fill_strategy(col, self.fill_vals[n])

@FillMissing
def encodes(self, to: TabularGPU):
    for n in to.cont_names:
        if n in self.na_dict:
            if self.add_col:
                to.items[n+'_na'] = to.items[n].isnull()
                if n+'_na' not in to.cat_names: to.cat_names.append(n+'_na')
            to.set_col(n, to.items[n].fillna(self.na_dict[n]))
        elif df[n].isnull().any():
            raise Exception(f"nan values in `{n}` but not in setup training set")

In [ ]:
fill1,fill2,fill3 = (FillMissing(fill_strategy=s) 
                     for s in [FillStrategy.median, FillStrategy.constant, FillStrategy.mode])
df = cudf.from_pandas(pd.DataFrame({'a':[0,1,np.nan,1,2,3,4]}))
df1 = df.copy(); df2 = df.copy()
to,to1,to2 = TabularGPU(df, cont_names='a'),TabularGPU(df1, cont_names='a'),TabularGPU(df2, cont_names='a')
fill1.setup(to); fill2.setup(to1); fill3.setup(to2)
test_eq(fill1.na_dict, {'a': 1.5})
test_eq(fill2.na_dict, {'a': 0})
test_eq(fill3.na_dict, {'a': 1.0})

for t in [to, to1, to2]: test_eq(t.cat_names, ['a_na'])

for to_,v in zip([to, to1, to2], [1.5, 0., 1.]):
    test_eq(to_.a.to_array(), np.array([0, 1, v, 1, 2, 3, 4]))
    test_eq(to_.a_na.to_array(), np.array([0, 0, 1, 0, 0, 0, 0]))
    
dfa = cudf.from_pandas(pd.DataFrame({'a':[np.nan,0,np.nan]}))
dfa1 = dfa.copy(); dfa2 = dfa.copy()
to,to1,to2 = TabularGPU(dfa, cont_names='a'),TabularGPU(dfa1, cont_names='a'),TabularGPU(dfa2, cont_names='a')
fill1(to); fill2(to1); fill3(to2)
for to_,v in zip([to, to1, to2], [1.5, 0., 1.]):
    test_eq(to_.a.to_array(), np.array([v, 0, v]))
    test_eq(to_.a_na.to_array(), np.array([1, 0, 1]))

## Tabular Pipelines -

In [ ]:
procs = [Normalize, Categorify, FillMissing, noop]
proc = Pipeline(procs)

#Test reordering and partialize
test_eq(L(proc.fs).mapped(type), [FillMissing, Transform, Categorify, Normalize])

df = cudf.from_pandas(pd.DataFrame({'a':[0,1,2,1,1,2,0], 'b':[0,1,np.nan,1,2,3,4]}))
to = TabularGPU(df, 'a', 'b')

#Test setup and apply on df_trn
proc.setup(to)
test_eq(to.a.to_array(), [1,2,3,2,2,3,1])
test_eq(to.b_na.to_array(), [1,1,2,1,1,1,1])
x = np.array([0,1,1.5,1,2,3,4])
m,s = x.mean(),x.std()
test_close(to.b.to_array(), (x-m)/s)
test_eq(proc.classes, {'a': ['#na#','0','1','2'], 'b_na': ['#na#','False','True']})

In [ ]:
#Test apply on y_names
procs = [Normalize, Categorify, FillMissing, noop]
proc = Pipeline(procs)

df = cudf.from_pandas(pd.DataFrame({'a':[0,1,2,1,1,2,0], 'b':[0,1,np.nan,1,2,3,4], 'c': ['b','a','b','a','a','b','a']}))
to = TabularGPU(df, 'a', 'b', y_names='c')

proc.setup(to)
test_eq(to.cat_names, ['a', 'b_na'])
test_eq(to.a.to_array(), [1,2,3,2,2,3,1])
test_eq(to.b_na.to_array(), [1,1,2,1,1,1,1])
test_eq(to.c.to_array(), [2,1,2,1,1,2,1])
x = np.array([0,1,1.5,1,2,3,4])
m,s = x.mean(),x.std()
test_close(to.b.to_array(), (x-m)/s)
test_eq(proc.classes, {'a': ['#na#','0','1','2'], 'b_na': ['#na#','False','True'], 'c': ['#na#','a','b']})

Pass the same `splits` as you will use for splitting the data, so that the setup is only done on the training set. `cat_names` are the names of the categorical variables, `cont_names` the continous ones, `cat_y` are the names of the dependent variables that are categories. If `inplace=True`, processing is applied inplace, otherwis it creates a copy of `df`.

In [ ]:
@delegates(TabularGPU)
def process_cudf(df, procs, inplace=True, splits=None, **kwargs):
    "Process `df` with `procs` and returns the processed dataframe and the `TabularProcessor` associated"
    df = df if inplace else df.copy()
    if splits is not None: df = df.iloc[sum(splits, [])].reset_index(drop=True)
    split = None if splits is None else len(splits[0])
    to = TabularGPU(df, split=split, **kwargs)
    proc = Pipeline(procs)
    proc.setup(to)
    return to,proc

In [ ]:
procs = [Normalize, Categorify, FillMissing, noop]
df = cudf.from_pandas(pd.DataFrame({'a':[0,1,2,1,1,2,0], 'b':[0,1,np.nan,1,2,3,4], 'c': ['b','a','b','a','a','b','a']}))
to,proc = process_cudf(df, procs, cat_names='a', cont_names='b', y_names='c', inplace=False)
test_eq(to.cat_names, ['a', 'b_na'])
test_eq(to.a.to_array(), [1,2,3,2,2,3,1])
test_eq(to.a.dtype,int)
test_eq(to.b_na.to_array(), [1,1,2,1,1,1,1])
test_eq(to.c.to_array(), [2,1,2,1,1,2,1])

In [ ]:
procs = [Normalize, Categorify, FillMissing, noop]
df = cudf.from_pandas(pd.DataFrame({'a':[0,1,2,1,1,2,0], 'b':[0,np.nan,1,1,2,3,4], 'c': ['b','a','b','a','a','b','a']}))
to,proc = process_cudf(df, procs, cat_names='a', cont_names='b', y_names='c', inplace=True, splits=[[0,1,4,6], [2,3,5]])
test_eq(to.cat_names, ['a', 'b_na'])
test_eq(to.a.to_array(), [1,2,2,1,0,2,0])
test_eq(to.a.dtype,int)
test_eq(to.b_na.to_array(), [1,2,1,1,1,1,1])
test_eq(to.c.to_array(), [2,1,1,1,2,1,2])

In [ ]:
#export
from torch.utils.dlpack import from_dlpack

@ReadTabBatch
def encodes(self, to: TabularGPU):
    return (from_dlpack(to.cats.to_dlpack()).long(),from_dlpack(to.conts.to_dlpack()).float()), from_dlpack(to.targ.to_dlpack()).long()

## Integration example

In [ ]:
path = untar_data(URLs.ADULT_SAMPLE)
df = cudf.from_pandas(pd.read_csv(path/'adult.csv'))
df_trn,df_tst = df.iloc[:10000].copy(),df.iloc[10000:].copy()
df_trn.head()

<cudf.DataFrame ncols=15 nrows=5 >

In [ ]:
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race']
cont_names = ['age', 'fnlwgt', 'education-num']
procs = [Categorify, FillMissing, Normalize]

splits = RandomSplitter()(range_of(df_trn))

In [ ]:
%time to,proc = process_cudf(df_trn, procs, splits=splits, cat_names=cat_names, cont_names=cont_names, y_names="salary")

CPU times: user 312 ms, sys: 24 ms, total: 336 ms
Wall time: 335 ms


In [ ]:
filts = [list(range(len(splits[0]))), list(range(len(splits[0]), 10000))]
dsrc = DataSource(to, filts=filts, tfms=[None])
dl = TabDataLoader(dsrc.valid, proc, bs=64, num_workers=0)

In [ ]:
dl.show_batch()

/home/sgugger/anaconda3/lib/python3.7/site-packages/cudf/io/dlpack.py:83: UserWarning: WARNING: cuDF to_dlpack() produces column-major (Fortran order) output. If the output tensor needs to be row major, transpose the output of this function.
  return cpp_dlpack.to_dlpack(gdf_cols)


,age,fnlwgt,education-num,workclass,education,marital-status,occupation,relationship,race,education-num_na,salary
0,48.000000,304791.001827,10.0,Private,Some-college,Widowed,Armed-Forces,Not-in-family,White,False,<50k
1,25.000000,202480.000000,12.0,?,Assoc-acdm,Never-married,Adm-clerical,Other-relative,White,False,<50k
2,50.000000,266944.997355,9.0,Private,HS-grad,Married-civ-spouse,Other-service,Wife,White,False,<50k
3,47.000000,109704.998693,9.0,Local-gov,HS-grad,Divorced,Farming-fishing,Not-in-family,White,False,<50k
4,20.999999,187087.999823,10.0,Private,Some-college,Never-married,Armed-Forces,Own-child,Other,False,<50k
5,26.000000,31143.001549,10.0,Private,Some-college,Never-married,Exec-managerial,Not-in-family,White,False,<50k
6,48.000000,133693.998794,13.0,Self-emp-not-inc,Bachelors,Married-spouse-absent,Farming-fishing,Not-in-family,Black,False,>=50k
7,36.000000,177907.000763,13.0,Private,Bachelors,Never-married,Priv-house-serv,Not-in-family,White,False,<50k
8,24.000001,241522.999018,9.0,Private,HS-grad,Married-civ-spouse,Priv-house-serv,Husband,White,False,>=50k
9,28.000000,412149.010278,6.0,Private,10th,Never-married,Handlers-cleaners,Other-relative,White,False,<50k


## Export -

In [ ]:
#hide
from local.notebook.export import notebook2script
notebook2script(all_fs=True)

Converted 00_test.ipynb.
Converted 01_core.ipynb.
Converted 01a_torch_core.ipynb.
Converted 01b_script.ipynb.
Converted 01c_dataloader.ipynb.
Converted 02_data_transforms.ipynb.
Converted 03_data_pipeline.ipynb.
Converted 05_data_core.ipynb.
Converted 06_data_source.ipynb.
Converted 07_vision_core.ipynb.
Converted 08_pets_tutorial.ipynb.
Converted 09_vision_augment.ipynb.
Converted 11_layers.ipynb.
Converted 11a_vision_models_xresnet.ipynb.
Converted 12_optimizer.ipynb.
Converted 13_learner.ipynb.
Converted 14_callback_schedule.ipynb.
Converted 15_callback_hook.ipynb.
Converted 16_callback_progress.ipynb.
Converted 17_callback_tracker.ipynb.
Converted 18_callback_fp16.ipynb.
Converted 19_callback_mixup.ipynb.
Converted 20_metrics.ipynb.
Converted 21_tutorial_imagenette.ipynb.
Converted 22_vision_learner.ipynb.
Converted 23_tutorial_transfer_learning.ipynb.
Converted 30_text_core.ipynb.
Converted 31_text_data.ipynb.
Converted 32_text_models_awdlstm.ipynb.
Converted 33_text_models_core.i